In [1]:
import pandas as pd
import numpy as np
from random import randrange
import numpy as np
from numpy.random import random_integers as rnd
import matplotlib.pyplot as plt
import math

In [2]:
def maze(width=80, height=40, complexity=.75, density =.75):
    # Only odd shapes
    shape = ((height//2)*2+1, (width//2)*2+1)
    # Adjust complexity and density relative to maze size
    complexity = int(complexity*(5*(shape[0]+shape[1])))
    density    = int(density*(shape[0]//2*shape[1]//2))
    # Build actual maze
    Z = np.zeros(shape)
    # Fill borders
    Z[0,:] = Z[-1,:] = 1
    Z[:,0] = Z[:,-1] = 1
    # Make isles
    for i in range(density):
        x, y = rnd(0,shape[1]//2)*2, rnd(0,shape[0]//2)*2
        Z[y,x] = 1
        for j in range(complexity):
            neighbours = []
            if x > 1:           neighbours.append( (y,x-2) )
            if x < shape[1]-2:  neighbours.append( (y,x+2) )
            if y > 1:           neighbours.append( (y-2,x) )
            if y < shape[0]-2:  neighbours.append( (y+2,x) )
            if len(neighbours):
                y_,x_ = neighbours[rnd(0,len(neighbours)-1)]
                if Z[y_,x_] == 0:
                    Z[y_,x_] = 1
                    Z[y_+(y-y_)//2, x_+(x-x_)//2] = 1
                    x, y = x_, y_
    return Z

In [3]:
def create_mz(width = 80, height = 40):
    mz = maze(width, height)
    for i in range(100):
        x = randrange(mz.shape[1]-1) #start 2
        y = randrange(mz.shape[1]-1) #end 3
        if (x!=0 & x!=(randrange(mz.shape[1]-1)) & y!=0 & y!=(randrange(mz.shape[1]-1)) &(mz[1][y]!=1) & (mz[mz.shape[0]-2][x]!=1)):
            mz[mz.shape[0]-1][x] = 2
            mz[0][y] = 3
            break
    start = (mz.shape[0]-1,x)
    end = (0, y)
    return(mz, start, end)

In [4]:
def moving(mz, seq, current_pos):
    for i in seq:
        if ((i == 'n') & (current_pos[0]!=0)):
            if((mz[current_pos[0]-1, current_pos[1]]!=1)):
                current_pos = (current_pos[0]-1, current_pos[1])
        if ((i == 's') & (current_pos[0]!=(mz.shape[0]-1))):
            if((mz[current_pos[0] + 1, current_pos[1]]!=1)):
                current_pos = (current_pos[0]+1, current_pos[1])
        if ((i == 'w') & (mz[(current_pos[0], current_pos[1]-1)]!=1)):
            current_pos = (current_pos[0], current_pos[1]-1)
        if ((i == 'e') & (mz[(current_pos[0], current_pos[1]+1)]!=1)):
            current_pos = (current_pos[0], current_pos[1]+1)
    return(current_pos)

In [5]:
def display_current_pos(mz, current_pos, figsize=(6,4), cmap = plt.cm.binary):
    mz2 = mz.copy()
    mz2[current_pos] = 4
    plt.figure(figsize=figsize)
    plt.imshow(mz2,interpolation='nearest', cmap=cmap)
    plt.xticks([]),plt.yticks([])
    plt.show()

In [6]:
def display_action_sequence(mz, seq, current_pos, figsize = (6,4), cmap = plt.cm.binary):
    mz2 = mz.copy()
    for i in seq:
        current_pos = moving(mz, i, current_pos)
        mz2[current_pos] = 4
    plt.figure(figsize=figsize)
    plt.imshow(mz2,interpolation='nearest', cmap=cmap) #plt.cm.binary
    plt.xticks([]),plt.yticks([])
    plt.show()

# Hybrid Ant Colony with Backtracking

In [7]:
mz, start, end = create_mz(width=10,height=10)

C:\Users\giorgio1\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. Please call randint(0, 5 + 1) instead
  
C:\Users\giorgio1\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead
C:\Users\giorgio1\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: This function is deprecated. Please call randint(0, 2 + 1) instead


In [8]:
def actions_val(gridWorld, ant_pos, start, end):
    #print(ant_pos)
    #print(start)
    value_act = []
    if (ant_pos == start):
        #print('ok')
        value_act = [0, float('nan'), float('nan'), float('nan')]
    elif (ant_pos == end):
        value_act = [float('nan'), 0, float('nan'), float('nan')]
    else:
        for i in actions:
            if (i == "n"):
                value_act.append(gridWorld[(ant_pos[0]-1, ant_pos[1])])
            if (i == "s"):
                value_act.append(gridWorld[(ant_pos[0]+1, ant_pos[1])])
            if (i == "e"):
                value_act.append(gridWorld[(ant_pos[0], ant_pos[1]+1)])
            if (i == "w"):
                value_act.append(gridWorld[(ant_pos[0], ant_pos[1]-1)])  
    return(value_act)

In [9]:
def is_decision_node(gridWorld, ant_pos, start, end, ant):
    b = False
    if (ant_pos == start):
        b = True
    if (ant_pos == end):
        b = True
    value_act = actions_val(gridWorld, ant_pos, start, end)
    non_zeros = np.count_nonzero(value_act)
    if (non_zeros <= 2):
        b = True
    how_many_ants = len(actions) - non_zeros
    if(ant['n_visited_nodes']==1)&(gridWorld[ant_pos]>900):
        b = True
    return b, how_many_ants

In [10]:
def moving_ant(gridWorld, ant_pos, start, end, ant):
    nm = np.nanmin(actions_val(gridWorld,ant_pos, start, end))
    if ((ant['n_visited_nodes']==1) & (np.nanmax(actions_val(gridWorld,ant_pos, start, end)) > 900) & (nm!=0)):
        a = np.nanargmax(actions_val(gridWorld,ant_pos, start, end))
        if(ant['start_node']!=moving(mz, seq = actions[a], current_pos = ant_pos )):
            action_idx = a
                #print(actions_val(gridWorld,ant_pos, start, end), actions[action_idx], ant_pos)
        else:
            action_idx = np.nanargmin(actions_val(gridWorld,ant_pos, start, end))
    else:
        action_idx = np.nanargmin(actions_val(gridWorld,ant_pos, start, end))
    #print(ant_pos, actions_val(gridWorld,ant_pos, start, end), ant['n_visited_nodes'], np.nanmax(actions_val(gridWorld,ant_pos, start, end)), np.nanmin(actions_val(gridWorld,ant_pos, start, end)), actions[action_idx])
    return(actions[action_idx]) 

In [11]:
def update_grid_after_action(gridWorld, ant_pos):
    if(gridWorld[ant_pos]==0):
        gridWorld[ant_pos] = 1
    return gridWorld

In [12]:
def training (gridWorld, max_iter = 200, start = start, end = end):
    n_ants = 0
    ants = {}
    ant_info = {}
    ant_info['position'] = start
    ant_info['start_node'] = start
    ant_info['n_visited_nodes'] = 1 #visto che parte da start che è un nodo, allora n_visited_nodes è 1
    ant_info['actions'] = []
    ants["ant{0}".format(n_ants)] = ant_info
    for i in range(max_iter):
        #print(gridWorld,ants)
        for j in range(len(ants)):
            n_visited_nodes = ants["ant{0}".format(j)]['n_visited_nodes']
            if (n_visited_nodes < 2 ):
                ant_pos = ants["ant{0}".format(j)]['position']
                action = moving_ant(gridWorld, ant_pos, start, end, ants["ant{0}".format(j)])
                ants["ant{0}".format(j)]['actions'].append(action)
                ants["ant{0}".format(j)]['position'] = moving(mz, seq = action, current_pos = ant_pos)
                ant_pos = ants["ant{0}".format(j)]['position']
                gridWorld = update_grid_after_action(gridWorld, ant_pos)
                b, how_many_ants = is_decision_node(gridWorld,ant_pos, start, end, ants["ant{0}".format(j)])
                if (b == True):
                    gridWorld[ant_pos] = 999
                    ants["ant{0}".format(j)]['n_visited_nodes'] += 1
                    if(ants["ant{0}".format(j)]['n_visited_nodes'] == 2):
                        ants["ant{0}".format(j)]['end_node'] = ant_pos
                    for i in range(how_many_ants):
                        n_ants += 1
                        ant_info = {}
                        ant_info['position'] = ant_pos
                        ant_info['n_visited_nodes'] = 1
                        ant_info['start_node'] = ant_pos
                        ant_info['actions'] = []
                        ants["ant{0}".format(n_ants)] = ant_info
            gridWorld = gridWorld * 0.9999 #evaporazione del feromone
            if (ant_pos == end):
                break
    return gridWorld, ants

In [13]:
def best_move(gridWorld, ant_pos, start, end):
    value_act = actions_val(gridWorld, ant_pos, start, end)
    #print(ant_pos, gridWorld, value_act)
    #print(gridWorld)
    for i in range(len(value_act)):
        #print( type(value_act[i]), value_act[i] )
        #print(value_act[i], ant_pos, value_act)
        if(value_act[i] > 200): #se è un decision node
            if (i == 0):
                pos = moving(mz, seq = 'n', current_pos = ant_pos)
                if (np.nanmax(actions_val(gridWorld, pos, start, end)) > 200): # se c'è un altro decision node
                    index = np.nanargmax(actions_val(gridWorld, pos, start, end)) #indice dell'azione che mi porta nel decision node
                    p = moving(mz, seq = actions[index], current_pos = ant_pos) #posizione in cui si trova il decision node
                    min_p = np.nanmin(actions_val(gridWorld, p, start, end))
                    if (min_p < np.nanmin(actions_val(gridWorld, pos, start, end))):
                        value_act[i] = index
                    else:
                        value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
                else:
                    value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
            if (i == 1):
                pos = moving(mz, seq = 's', current_pos = ant_pos)
                if (np.nanmax(actions_val(gridWorld, pos, start, end)) > 200): # se c'è un altro decision node
                    index = np.nanargmax(actions_val(gridWorld, pos, start, end)) #indice dell'azione che mi porta nel decision node
                    p = moving(mz, seq = actions[index], current_pos = ant_pos) #posizione in cui si trova il decision node
                    min_p = np.nanmin(actions_val(gridWorld, p, start, end))
                    if (min_p < np.nanmin(actions_val(gridWorld, pos, start, end))):
                        value_act[i] = index
                    else:
                        value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
                else:
                    value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
            if (i == 2):
                pos = moving(mz, seq = 'e', current_pos = ant_pos)
                if (np.nanmax(actions_val(gridWorld, pos, start, end)) > 200): # se c'è un altro decision node
                    index = np.nanargmax(actions_val(gridWorld, pos, start, end)) #indice dell'azione che mi porta nel decision node
                    p = moving(mz, seq = actions[index], current_pos = ant_pos) #posizione in cui si trova il decision node
                    min_p = np.nanmin(actions_val(gridWorld, p, start, end))
                    if (min_p < np.nanmin(actions_val(gridWorld, pos, start, end))):
                        value_act[i] = index
                    else:
                        value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
                else:
                    value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
            if (i == 3):
                pos = moving(mz, seq = 'w', current_pos = ant_pos)
                if (np.nanmax(actions_val(gridWorld, pos, start, end)) > 200): # se c'è un altro decision node
                    index = np.nanargmax(actions_val(gridWorld, pos, start, end)) #indice dell'azione che mi porta nel decision node
                    p = moving(mz, seq = actions[index], current_pos = ant_pos) #posizione in cui si trova il decision node
                    min_p = np.nanmin(actions_val(gridWorld, p, start, end))
                    if (min_p < np.nanmin(actions_val(gridWorld, pos, start, end))):
                        value_act[i] = index
                    else:
                        value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
                else:
                    value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
    #if (i == 3):
        #pos = moving(mz, seq = 'w', current_pos = ant_pos)
        #value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
    #print(ant_pos, value_act)
    action_idx = np.nanargmin(value_act)
    return(actions[action_idx])

def bm(gridWorld, ant_pos, start, end):
    value_act = actions_val(gridWorld, ant_pos, start, end)
    #print(ant_pos, gridWorld, value_act)
    #print(gridWorld)
    for i in range(len(value_act)):
        #print( type(value_act[i]), value_act[i] )
        #print(value_act[i], ant_pos, value_act)
        if(value_act[i] > 200): #se è un decision node
            if (i == 0):
                pos = moving(mz, seq = 'n', current_pos = ant_pos)
                value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
            if (i == 1):
                pos = moving(mz, seq = 's', current_pos = ant_pos)
                value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
            if (i == 2):
                pos = moving(mz, seq = 'e', current_pos = ant_pos)
                value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
            if (i == 3):
                pos = moving(mz, seq = 'w', current_pos = ant_pos)
                value_act[i] = np.nanargmin(actions_val(gridWorld, pos, start, end))
    action_idx = np.nanargmin(value_act)
    return(actions[action_idx])

In [14]:
def inverse_action_sequence(seq):
    seq = seq[::-1]
    out =[]
    for i in seq:
        if (i == "n"):
            out.append('s')
        if (i == "s"):
            out.append('n')
        if (i == "e"):
            out.append('w')
        if (i == "w"):
            out.append('e')
    return out

In [15]:
def best_seq(gridWorld, start, end):
    gridWorld = prepare_grid(gridWorld)
    gridWorld[start] = 0
    end_ = False
    ant_pos = end
    best_moves = []
    while (end_!=True): #for i in range(40): #
        #print(best_moves)
        #gridWorld[ant_pos] = 50 #messa ora
        action = best_move(gridWorld, ant_pos, start, end)
        best_moves.append(action)
        gridWorld[ant_pos] = 50 
        ant_pos = moving(mz, seq = action, current_pos = ant_pos)
        if (ant_pos == start):
            break
    best_moves.append(ant_pos)
    best_moves = inverse_action_sequence(best_moves)
    return(best_moves)

In [16]:
def prepare_grid(gridWorld):
    gridWorld[gridWorld==0] = 15 #per cercare la soluzione ottimale non passare sopra le celle con valore 0
    nodes = gridWorld[gridWorld>50]
    pos_nodes = []
    for i in nodes:
        x,y = np.where(gridWorld == i)
        pos_nodes.append((int(x),int(y)))
    for i in pos_nodes:
        value_act = actions_val(gridWorld, i, start, end)
        min_v_act = np.nanmin(value_act)
        if(np.nanmax(value_act)>200):
            index = np.nanargmax(actions_val(gridWorld, i, start, end)) #indice dell'azione che mi porta nel decision node
            p = moving(mz, seq = actions[index], current_pos = i) #posizione in cui si trova il decision node
            min_p = np.nanmin(actions_val(gridWorld, p, start, end))
            if (min_p < np.nanmin(actions_val(gridWorld, i, start, end))):
                gridWorld[i] = min_p
            else:
                gridWorld[i] = np.nanmin(actions_val(gridWorld, i, start, end))
        else:
            gridWorld[i] = np.nanmin(actions_val(gridWorld, i, start, end))
    return gridWorld

# Try the Algorithm

In [64]:
%%time
mz, start, end = create_mz(width=500,height=500)
actions = ['n', 's', 'e','w']
gridWorld = mz.copy()
gridWorld[gridWorld == 1] = float('nan')
gridWorld[gridWorld == 0] = 0
gridWorld[gridWorld == 2] = 0
gridWorld[gridWorld == 3] = 0
gridWorld[start]=0
gridWorld[end]=0

C:\Users\giorgio1\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. Please call randint(0, 250 + 1) instead
  
C:\Users\giorgio1\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead
C:\Users\giorgio1\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: This function is deprecated. Please call randint(0, 2 + 1) instead
C:\Users\giorgio1\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead


Wall time: 12min 46s


In [65]:
%%time
gridWorld, ants = training(gridWorld, max_iter= len(gridWorld[gridWorld==0]), start=start, end = end)

Wall time: 2h 2min 14s


In [66]:
len(gridWorld[gridWorld==0])

21966

In [ ]:
s = best_seq(gridWorld=gridWorld, start=start, end=end)

C:\Users\giorgio1\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
display_action_sequence(mz, s, start, figsize=(8,8))